In [4]:
import cv2
import mediapipe as mp
import json
import os
import numpy as np

# Input and output directories
input_dir = "./resistive_band_person_16/"
# input_dir = "./my_resistive_test/"    # another tests
output_dir = "./output/"
keypoints_dir = "./keypoints/"

# Create output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(keypoints_dir, exist_ok=True)

# Initialize MediaPipe Pose and Face Detection
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_detection = mp.solutions.face_detection

# Define function to blur a keypoint
def blur_keypoint(image, x, y, radius=30):
    h, w, _ = image.shape
    cx, cy = int(x * w), int(y * h)  # Convert normalized coordinates to pixel positions
    
    if 0 <= cx < w and 0 <= cy < h:
        # Define the region of interest (ROI) around the keypoint with a square area (using the radius)
        top_left_x = max(0, cx - radius)
        top_left_y = max(0, cy - radius)
        bottom_right_x = min(w, cx + radius)
        bottom_right_y = min(h, cy + radius)
        
        # Extract the region of interest (ROI) around the keypoint
        roi = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        
        # Apply Gaussian blur to the ROI
        blurred_roi = cv2.GaussianBlur(roi, (0, 0), sigmaX=radius, sigmaY=radius)
        
        # Place the blurred ROI back into the image
        image[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = blurred_roi


# Process all images in the input directory
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    for filename in os.listdir(input_dir):
        face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Load the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process the image to extract pose
            results = pose.process(image_rgb)
            face_results = face_detection.process(image_rgb)

            if results.pose_landmarks:
                # Blur head keypoints (e.g., Nose, Left Eye, Right Eye, Left Ear, Right Ear)
                head_keypoints = [
                    mp_pose.PoseLandmark.NOSE,
                    # mp_pose.PoseLandmark.LEFT_EYE,
                    # mp_pose.PoseLandmark.RIGHT_EYE,
                    # mp_pose.PoseLandmark.LEFT_EAR,
                    # mp_pose.PoseLandmark.RIGHT_EAR,
                    # mp_pose.PoseLandmark.MOUTH_LEFT,
                    # mp_pose.PoseLandmark.MOUTH_RIGHT
                ]
                for keypoint in head_keypoints:
                    kp = results.pose_landmarks.landmark[keypoint]
                    blur_keypoint(image, kp.x, kp.y)

                # Draw the skeleton on the image
                annotated_image = image.copy()
                mp_drawing.draw_landmarks(
                    annotated_image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                )

                # Save the annotated image
                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, annotated_image)

                # Save keypoints to JSON
                keypoints = {}
                for idx, landmark in enumerate(results.pose_landmarks.landmark):
                    keypoints[idx] = {
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    }

                keypoints_path = os.path.join(keypoints_dir, f"{os.path.splitext(filename)[0]}.json")
                with open(keypoints_path, "w") as json_file:
                    json.dump(keypoints, json_file, indent=4)


I0000 00:00:1736183641.593035   33601 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736183641.597630   37848 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
I0000 00:00:1736183641.601734   33601 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1736183641.606403   37861 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1736183641.612468   37852 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736183641.656200   37840 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736183641.676920   37845 inference_feedback_manager.cc:114] Feedback manager requires a model with a sing